## Importing Data and Libraries

In [1]:
%tensorflow_version 2.x
import tensorflow as tf
print("Tensorflow version " + tf.__version__)

try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

Tensorflow version 2.4.1
Running on TPU  ['10.21.139.146:8470']
INFO:tensorflow:Initializing the TPU system: grpc://10.21.139.146:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.21.139.146:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [2]:
import os
import numpy as np
import string
import itertools
from tqdm.notebook import tqdm
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, Sequential, callbacks, utils
from nltk.translate.bleu_score import SmoothingFunction, corpus_bleu, sentence_bleu

In [3]:
path = '/content/drive/MyDrive/Next word Generation/Data/train.txt'
data = ''
with open(path, 'r', encoding='utf-8') as file:
  data = file.readlines()

## Preprocessing

In [4]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [5]:
def tokenize(data, to_select_lines=-1):

  result = []
  table = str.maketrans('', '', string.punctuation+'|।')

  idx = 0

  for line in data:
    line_tokens = []
    idx += 1
    if (to_select_lines == -1):
      idx -= 1
    if (idx == to_select_lines):
      break

    for word in line.split():
      if len(word.translate(table)) > 2:
        line_tokens.append(word.translate(table))

    result.append(line_tokens)

  return result

In [6]:
def create_dict(tokens, to_select_words=-1):

  word_dict = {}
  for i in tokens:
    for j in i:
      try:
        word_dict[j] += 1
      except:
        word_dict[j] = 1

  res_dict = {k: v for k, v in sorted(word_dict.items(), key=lambda item: item[1], reverse=True)}


  if (to_select_words == -1):
    final_dict = {}
    for idx, (key, value) in zip(range(len(res_dict)), res_dict.items()):
      final_dict[key] = idx
    return final_dict

  final_dict = {}
  for idx, (key, value) in zip(range(to_select_words), res_dict.items()):
    final_dict[key] = idx

  return final_dict

In [7]:
def convert(line, word_dict):
  result = []
  for word in line:
    try:
      result.append(word_dict[word])
    except:
      result.append(0)

  return result

In [8]:
def get_sequences(data, word_dict, n_gram_length):

  input_sequence = []
  idx = 0

  for line in data:
    token_list = convert(line, word_dict)
    for i in range(1, len(token_list)):
      n_gram_seq = token_list[i:i+n_gram_length]
      if (len(n_gram_seq) == n_gram_length):
        input_sequence.append(n_gram_seq)

  return input_sequence

In [9]:
def preprocess(data, to_select_lines=-1, to_select_words=-1, n_gram_length=3):

  print("Tokenizing")
  tokenized = tokenize(data, to_select_lines)
  print("Creating Dictionary")
  word_dict = create_dict(tokenized, to_select_words)
  print("Creating Input Sequences")
  input_sequences = get_sequences(tokenized, word_dict, n_gram_length)

  del tokenized

  return word_dict, input_sequences

## Changing data for input

In [10]:
word_dict, input_sequences = preprocess(data, to_select_words=10000, n_gram_length=3)

Tokenizing
Creating Dictionary
Creating Input Sequences


In [11]:
reverse_dict = {}
for key, value in word_dict.items():
  reverse_dict[value] = key

In [12]:
len(input_sequences), len(word_dict)

(1287448, 10000)

In [13]:
input_sequences = np.array(input_sequences)
np.random.shuffle(input_sequences)
input_sequences[:3]

array([[ 339,   51,  322],
       [1189, 2987, 4872],
       [2072,   73,  174]])

In [ ]:
input_sequences.shape, len(word_dict)

((1287448, 3), 10000)

In [ ]:
def split_x_label(train):
  x = train[:-1]
  label = train[-1]
  label = tf.one_hot(label, len(word_dict))
  return x, label

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices(input_sequences)

In [ ]:
dataset = dataset.map(split_x_label)

In [ ]:
dataset = dataset.batch(256, drop_remainder=True)

## Saving the dictionary and sequence

In [ ]:
a = pd.DataFrame(data = word_dict.keys())
a['values'] = word_dict.values()
a.columns = ['words', 'values']
a.head()
a.to_csv('/content/drive/MyDrive/Next word Generation/Dictionary/entry_1.csv', index=False)

In [ ]:
np.save('/content/drive/MyDrive/Next word Generation/Sequences/entry_1.npy', input_sequences)

## Creating Model

In [ ]:
def create_model(word_dict):

  model = Sequential([
                    layers.Embedding(len(word_dict), 64),
                    layers.Bidirectional(layers.LSTM(512)),
                    layers.Dense(len(word_dict), activation='softmax')
          ])
  
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

  return model

## Training the model

In [ ]:
def check_results(input_string, model, word_dict, reverse_dict, generation_len=10):

  test_input = input_string
  next_words = generation_len
  table = str.maketrans('', '', string.punctuation)

  for i in range(next_words):
    token_list = [w.translate(table) for w in test_input.split() if w.translate(table) != '']
    token_list = convert(token_list, word_dict)
    token_list = [token_list[-2:]]
    predicted =  model.predict_classes(token_list, verbose=0)[0]
    
    output_word = ''
    if predicted in reverse_dict:
      output_word = reverse_dict[predicted]
    test_input += " " + output_word

    print(test_input)

In [ ]:
model = create_model(word_dict)

In [ ]:
# model = tf.keras.models.load_model('/content/drive/MyDrive/Next word Generation/Models/small_model.h5')

In [ ]:
save_point = callbacks.ModelCheckpoint('/content/drive/MyDrive/Next word Generation/Models/entry_1.h5', monitor='loss', save_best_only=True)
stop_check = callbacks.EarlyStopping(
                monitor='loss', min_delta=0, patience=4, verbose=0,
                mode='auto', baseline=None, restore_best_weights=False
              )

In [ ]:
model.fit(dataset.repeat(), epochs=500, steps_per_epoch=500, callbacks=[save_point])

Epoch 1/500
500/500 [==============================] - 32s 57ms/step - loss: 7.6850 - accuracy: 0.1347
Epoch 2/500
500/500 [==============================] - 29s 58ms/step - loss: 6.9554 - accuracy: 0.1492
Epoch 3/500
500/500 [==============================] - 29s 58ms/step - loss: 6.6115 - accuracy: 0.1667
Epoch 4/500
500/500 [==============================] - 29s 58ms/step - loss: 6.3991 - accuracy: 0.1768
Epoch 5/500
500/500 [==============================] - 29s 58ms/step - loss: 6.2108 - accuracy: 0.1841
Epoch 6/500
500/500 [==============================] - 29s 58ms/step - loss: 6.0390 - accuracy: 0.1914
Epoch 7/500
500/500 [==============================] - 29s 57ms/step - loss: 5.8887 - accuracy: 0.1972
Epoch 8/500
500/500 [==============================] - 29s 59ms/step - loss: 5.7772 - accuracy: 0.2023
Epoch 9/500
500/500 [==============================] - 29s 57ms/step - loss: 5.6717 - accuracy: 0.2054
Epoch 10/500
500/500 [==============================] - 29s 57ms/step - l

KeyboardInterrupt: ignored

## Functions for Evaluation Metrics

In [ ]:
def bleu(ref, gen):
    ''' 
    calculate pair wise bleu score. uses nltk implementation
    Args:
        references : a list of reference sentences 
        candidates : a list of candidate(generated) sentences
    Returns:
        bleu score(float)
    '''
    ref_bleu = []
    gen_bleu = []
    for l in gen:
        gen_bleu.append(l.split())
    for i,l in enumerate(ref):
        ref_bleu.append([l.split()])
    cc = SmoothingFunction()
    score_bleu = corpus_bleu(ref_bleu, gen_bleu, weights=(0, 1, 0, 0), smoothing_function=cc.method4)
    return score_bleu

In [ ]:
def _split_into_words(sentences):
  """Splits multiple sentences into words and flattens the result"""
  return list(itertools.chain(*[_.split(" ") for _ in sentences]))

#supporting function
def _get_word_ngrams(n, sentences):
  """Calculates word n-grams for multiple sentences.
  """
  assert len(sentences) > 0
  assert n > 0

  words = _split_into_words(sentences)
  return _get_ngrams(n, words)

#supporting function
def _get_ngrams(n, text):
  """Calcualtes n-grams.
  Args:
    n: which n-grams to calculate
    text: An array of tokens
  Returns:
    A set of n-grams
  """
  ngram_set = set()
  text_length = len(text)
  max_index_ngram_start = text_length - n
  for i in range(max_index_ngram_start + 1):
    ngram_set.add(tuple(text[i:i + n]))
  return ngram_set

def rouge_n(reference_sentences, evaluated_sentences, n=2):
  """
  Computes ROUGE-N of two text collections of sentences.
  Source: http://research.microsoft.com/en-us/um/people/cyl/download/
  papers/rouge-working-note-v1.3.1.pdf
  Args:
    evaluated_sentences: The sentences that have been picked by the summarizer
    reference_sentences: The sentences from the referene set
    n: Size of ngram.  Defaults to 2.
  Returns:
    recall rouge score(float)
  Raises:
    ValueError: raises exception if a param has len <= 0
  """
  if len(evaluated_sentences) <= 0 or len(reference_sentences) <= 0:
    raise ValueError("Collections must contain at least 1 sentence.")

  evaluated_ngrams = _get_word_ngrams(n, evaluated_sentences)
  reference_ngrams = _get_word_ngrams(n, reference_sentences)
  reference_count = len(reference_ngrams)
  evaluated_count = len(evaluated_ngrams)

  # Gets the overlapping ngrams between evaluated and reference
  overlapping_ngrams = evaluated_ngrams.intersection(reference_ngrams)
  overlapping_count = len(overlapping_ngrams)

  # Handle edge case. This isn't mathematically correct, but it's good enough
  if evaluated_count == 0:
    precision = 0.0
  else:
    precision = overlapping_count / evaluated_count

  if reference_count == 0:
    recall = 0.0
  else:
    recall = overlapping_count / reference_count

  f1_score = 2.0 * ((precision * recall) / (precision + recall + 1e-8))

  #just returning recall count in rouge, useful for our purpose
  return recall

In [ ]:
def calculate_perplexity(predictions, labels):
  log_probabilities = []
  for i in range(0, len(predictions)):
    log_prob = np.log(np.sum(predictions[i] * labels[i]))
    log_probabilities.append(log_prob)

  sum_log_probabilities = np.sum(log_probabilities)
  per_char_log_probability = sum_log_probabilities / len(predictions)
  perplexity = np.exp(per_char_log_probability)
  return perplexity

## Result of Evaluation Metrics

## Visualizing the results

In [ ]:
path = '/content/drive/MyDrive/Next word Generation/Data/train.txt'
data = ''
with open(path, 'r', encoding='utf-8') as file:
  data = file.readlines()

In [ ]:
data[50:55]

["ਇਸ ਤੋਂ ਪਹਿਲਾਂ ਸਨਿਚਰਵਾਰ ਭਾਰਤ ਸਰਕਾਰ ਨੇ ਕਿਹਾ ਕਿ ਉਹ ਪਾਕਿਸਤਾਨੀ ਸੈਨਿਕਾਂ ਵਲੋਂ ਦੋ ਭਾਰਤੀ ਸੈਨਿਕਾਂ ਨੂੰ ਮਾਰ ਕੇ ਉਨ੍ਹਾਂ ਦਾ ਸਿਰ ਕਲਮ ਕਰਕੇ ਲਿਜਾਣ ਵਿਰੁੱਧ ਬਦਲੇ ਦੀ ਕਰਵਾਈ ਨਹੀਂ ਕਰੇਗਾ ਪਰ ਏਅਰ ਚੀਫ ਮਾਰਸ਼ਲ ਨਾਰਮਨ ਅਨਿਲ ਕੁਮਾਰ ਬਰਾਉਨ ਨੇ ਕਲ੍ਹ ਕਿਹਾ ਸੀ ਕਿ 2003 ਤੋਂ ਕੰਟਰੋਲ ਰੇਖਾ 'ਤੇ ਚਲੀ ਆ ਰਹੀ ਜੰਗਬੰਦੀ ਦੀ ਪਾਕਿ ਵਲੋਂ ਉਲੰਘਣਾ ਕਦੇ ਵੀ ਬਰਦਾਸ਼ਤ ਨਹੀਂ ਕੀਤੀ ਜਵੇਗੀ |\n",
 "ਅਸੀਂ ਸਥਿਤੀ 'ਤੇ ਸਾਵਧਾਨੀ ਨਾਲ ਨਜ਼ਰ ਰੱਖ ਰਹੇ ਹਾਂ |\n",
 'ਜੇਕਰ ਉਲੰਘਣਾ ਜਾਰੀ ਰਾਹੀ ਤਾਂ ਅਸੀਂ ਦੂਸਰੇ ਬਦਲ ਬਾਰੇ ਸੋਚਾਂਗੇ ਪਰ ਉਨ੍ਹਾਂ ਦੂਸਰੇ ਬਦਲ ਬਾਰੇ ਦੱਸਣ ਤੋਂ ਇਨਕਾਰ ਕਰ ਦਿੱਤਾ ਸੀ |\n',
 "ਜੰਮੂ ਤੇ ਕਸ਼ਮੀਰ ਵਿਚ ਭਾਰਤੀ ਫ਼ੌਜ ਵਲੋਂ ਕਥਿਤ ਰੂਪ ਵਿਚ ਕੀਤੀ ਗੋਲੀਬਾਰੀ ਵਿਚ ਪਾਕਿਸਤਾਨੀ ਸੈਨਿਕ ਦੇ ਮਾਰੇ ਜਣ ਪਿੱਛੋਂ 6 ਜਨਵਰੀ ਤੋਂ ਹੀ ਭਾਰਤ ਅਤੇ ਪਾਕਿ ਵਿਚਕਾਰ ਸਰਹੱਦ 'ਤੇ ਤਣਾਅ ਬਣਿਆਂ ਹੋਇਆ ਹੈ |\n",
 'ਦੋ ਦਿਨ ਪਿੱਛੋਂ ਪਾਕਿ ਸੈਨਿਕ ਪੁਣਛ ਜ਼ਿਲ੍ਹੇ ਵਿਚ ਮੇਂਧਰ ਸੈਕਟਰ ਵਿਚ ਦੋ ਭਾਰਤੀ ਸੈਨਿਕਾਂ ਦਾ ਸਿਰ ਵੱਢ ਕੇ ਲੈ ਗਏ |\n']

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Next word Generation/Dictionary/word_dict_2.csv')
model = tf.keras.models.load_model('/content/drive/MyDrive/Next word Generation/Models/M2.h5')

In [ ]:
word_dict = {}
for i,j in zip(df['words'], df['values']):
  word_dict[i] = j

In [ ]:
reverse_dict = {}
for key, value in word_dict.items():
  reverse_dict[value] = key

In [ ]:
table = str.maketrans('', '', string.punctuation+'|।')

In [ ]:
def convert(line, word_dict):
  result = []
  for word in line:
    try:
      result.append(word_dict[word])
    except:
      result.append(0)
  return result

In [ ]:
test_input = 'ਮੇਰੀ ਇੰਡੀਆ'
next_words = 10
array = np.zeros((10, len(word_dict)))

for i in range(next_words):

  token_list = [w.translate(table) for w in test_input.split() if w.translate(table) != '']
  token_list = convert(token_list, word_dict)
  token_list = [token_list[-2:]]

  array[i] = model.predict(token_list, verbose=0)[0]

  values = np.argsort(array[i])

  pred_words = []
  for x,y in zip(range(5), values[::-1]):
    pred_words.append(reverse_dict[y])
    print(x, "  ", reverse_dict[y], " ", array[i][y])

  selected = int(input('Enter the index of the selected word.\n'))

  if selected == 0:
    test_input += " " + pred_words[0]
  elif selected == 1:
    test_input += " " + pred_words[1]
  elif selected == 2:
    test_input += " " + pred_words[2]
  elif selected == 3:
    test_input += " " + pred_words[3]
  elif selected == 4:
    test_input += " " + pred_words[4]


  print(test_input)

0    ਨੂੰ   0.29495668411254883
1    ਵਾਪਸ   0.14238789677619934
2    ਮੇਰੇ   0.12235148251056671
3    ਬਣਦੀ   0.05054636672139168
4    ਨੇੜੇ   0.042084213346242905
Enter the index of the selected word.
2
ਮੇਰੀ ਇੰਡੀਆ ਮੇਰੇ
0    ਨੂੰ   0.9782037734985352
1    ਵੱਲੋਂ   0.0049968259409070015
2    ਤਿੰਨ   0.0017017634818330407
3    ਕੋਲ   0.0016132036689668894
4    ਅੰਦਰ   0.0014818258350715041
Enter the index of the selected word.
4
ਮੇਰੀ ਇੰਡੀਆ ਮੇਰੇ ਅੰਦਰ
0    ਅਜੇ   0.2580052316188812
1    ਮੈਨੂੰ   0.07176513969898224
2    ਨੂੰ   0.0672178640961647
3    ਹੁਣ   0.06109967455267906
4    ਬਿਨ੍ਹਾਂ   0.05984465777873993
